In [41]:
#coding:utf8
import pandas as pd
import numpy as np

In [56]:
dir_path = r'E:/企点工作/20171011 非法企业售前识别/'
srcfile = dir_path + '/bad_uin_pv.csv'
pv = pd.read_csv(srcfile.decode('utf8').encode('gbk'))
pv.drop_duplicates(inplace=True)
pv.drop(['qq','f_create_day','product','is_bad_uin'], axis=1, inplace=True)
print pv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 676 entries, 0 to 675
Data columns (total 10 columns):
uin                    676 non-null int64
visit_day_num          676 non-null int64
pv_num                 676 non-null int64
session_num            676 non-null int64
url_num                676 non-null int64
viewtime_7             676 non-null int64
last_visit_day         676 non-null int64
is_pv_apply            676 non-null int64
is_pv_apply_sucdess    676 non-null int64
is_online_buy          676 non-null int64
dtypes: int64(10)
memory usage: 58.1 KB
None


In [43]:
srcfile = dir_path + '/res_interal_bad_uin_20171012'
face = pd.read_csv(srcfile.decode('utf8').encode('gbk'), header=None, names=['qq','age','edu','income','sex','city'])
face.drop_duplicates(inplace=True)
print face.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1428 entries, 0 to 1877
Data columns (total 6 columns):
qq        1428 non-null int64
age       1428 non-null int64
edu       1428 non-null int64
income    1428 non-null int64
sex       1428 non-null int64
city      1428 non-null int64
dtypes: int64(6)
memory usage: 78.1 KB
None


In [44]:
srcfile = dir_path + '/inter_bad_uin_20171011'
bad_uin = pd.read_csv(srcfile.decode('utf8').encode('gbk'), sep='\t')
bad_uin.drop_duplicates(inplace=True)
print bad_uin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1878 entries, 0 to 1877
Data columns (total 5 columns):
f_uin           1878 non-null int64
f_create_day    1878 non-null int64
f_admin_qq      1878 non-null int64
f_product       1878 non-null object
f_bad_uin       1878 non-null int64
dtypes: int64(4), object(1)
memory usage: 88.0+ KB
None


In [45]:
len(pv['qq'].unique())

569

In [61]:
all = pd.merge(bad_uin, face, how='left', left_on='f_admin_qq', right_on='qq')

all = pd.merge(all, pv, how='left', left_on='f_uin', right_on='uin')
all = all.drop(['qq', 'uin'], axis=1)
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1878 entries, 0 to 1877
Data columns (total 19 columns):
f_uin                  1878 non-null int64
f_create_day           1878 non-null int64
f_admin_qq             1878 non-null int64
f_product              1878 non-null object
f_bad_uin              1878 non-null int64
age                    1878 non-null int64
edu                    1878 non-null int64
income                 1878 non-null int64
sex                    1878 non-null int64
city                   1878 non-null int64
visit_day_num          676 non-null float64
pv_num                 676 non-null float64
session_num            676 non-null float64
url_num                676 non-null float64
viewtime_7             676 non-null float64
last_visit_day         676 non-null float64
is_pv_apply            676 non-null float64
is_pv_apply_sucdess    676 non-null float64
is_online_buy          676 non-null float64
dtypes: float64(9), int64(9), object(1)
memory usage: 293.4+ KB


In [87]:
# 只要有一天访问数据，就算是有PV数据
all['is_has_pv'] = all['visit_day_num']>0
all['is_has_pv']

# 只要有一个画像数据，就算是有画像数据
all['is_has_face']  = np.any(all[['age','sex','edu','income','city']]>0, axis=1)
sum(all['is_has_face'])

# 看QQ是否重复 
all['is_qq_dup'] = 0
all.loc[ all['f_admin_qq'].duplicated(), 'is_qq_dup'] = 1

In [88]:
out_file = dir_path + '/all_bad_uin_20171011.csv'
all.to_csv(out_file.decode('utf8').encode('gbk'))

In [91]:
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1878 entries, 0 to 1877
Data columns (total 22 columns):
f_uin                  1878 non-null int64
f_create_day           1878 non-null int64
f_admin_qq             1878 non-null int64
f_product              1878 non-null object
f_bad_uin              1878 non-null int64
age                    1878 non-null int64
edu                    1878 non-null int64
income                 1878 non-null int64
sex                    1878 non-null int64
city                   1878 non-null int64
visit_day_num          676 non-null float64
pv_num                 676 non-null float64
session_num            676 non-null float64
url_num                676 non-null float64
viewtime_7             676 non-null float64
last_visit_day         676 non-null float64
is_pv_apply            676 non-null float64
is_pv_apply_sucdess    676 non-null float64
is_online_buy          676 non-null float64
is_has_pv              1878 non-null bool
is_has_face            1

In [99]:
total_num = all.shape[0]
total_has_pv = sum(all['is_has_pv'])
total_has_face = sum(all['is_has_face'])
total_bad_uin = sum(all['f_bad_uin'])
print('total_num:%d bad_uin:%d has_pv:%d has_face:%d'%(total_num, total_bad_uin, total_has_pv, total_has_face) )


total_num:1878 bad_uin:253 has_pv:676 has_face:1660


,f_uin,f_create_day,f_admin_qq,f_product,age,edu,income,sex,city,visit_day_num,...,session_num,url_num,viewtime_7,last_visit_day,is_pv_apply,is_pv_apply_sucdess,is_online_buy,is_has_pv,is_has_face,is_qq_dup
f_bad_uin,,,,,,,,,,,,,,,,,,,,,
0,1625,1625,1625,1625,1625,1625,1625,1625,1625,640,...,640,640,640,640,640,640,640,1625,1625,1625
1,253,253,253,253,253,253,253,253,253,36,...,36,36,36,36,36,36,36,253,253,253
